# Подготовка данных

## Импортируем необходимые библиотеки

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

PATH = '/kaggle/input/transformer-time/'

## Составляем X_data и y_data

In [ ]:
# Состоит из названия файла и времени до поломки
y_data = pd.read_csv(PATH + 'train.csv', index_col='id')
y_data.head()

In [ ]:
# Итерируем по строкам в датафрейме(кортеж - (it(имя в нашем случае файла, метки)))
# Загружаем в словарь только те данные, по которым есть время до поломки

X_data = {}
for row in y_data.iterrows():
    file_name = row[0]
    path = PATH + f'data_train/data_train/{file_name}'
    X_data[file_name] = pd.read_csv(path)

In [ ]:
X_data[file_name].shape

In [ ]:
X_data[file_name].head(2)

# Моделирование

# Добавляем необходимые библиотеки


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, f1_score
from catboost import CatBoostRegressor
from tsfresh.feature_extraction import extract_features, MinimalFCParameters

## Реализуем прогноз по среднему. В дальнейшем результат ниже нужно будет улучшить, чтобы было ясно, что новое решение лучше самого простого решения

In [ ]:
# Разделяем на тестовые обучающие данные

y_train, y_val = train_test_split(y_data, test_size=0.25, random_state=1)
print(f'Train set shape: {y_train.shape}')
print(f'Test set shape: {y_val.shape}')

In [ ]:
y_train_pred = [y_train.mean().values[0]] * len(y_train)
y_val_pred = [y_train.mean().values[0]] * len(y_val)

In [ ]:
# MAE на тренировочных
mae_train = round(mean_absolute_error(y_train, y_train_pred), 2)

# MAE на тестовых
mae_val = round(mean_absolute_error(y_val, y_val_pred), 2)

print(f'MAE on train set = {mae_train}')
print(f'MAE on test set = {mae_val}')

### Реализация улучшенной модели
Буду использовать следующие подходы: Агрегация данных с помощью TSFresh + Градиентный бустинг (CatBoost)

In [ ]:
X = pd.concat([X_data[file].assign(id=file) for file in y_data.index], axis=0, ignore_index=True)
y = y_data.copy()

from tsfresh.feature_extraction.settings import MinimalFCParameters, ComprehensiveFCParameters,EfficientFCParameters

# Выбираем EfficientFCParameters, т.к. он гораздо быстрее ComprehensiveFCParameters +
# в данной задаче MAE на ComprehensiveFCParameters и EfficientFCParameters в данном подходе почти равны
#(выявлено экспериментальным методом)

settings =EfficientFCParameters()

X = extract_features(X, 
                     column_id="id", 
                     default_fc_parameters=settings).loc[y.index]

# Импортируем необходимые библиотеки

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Для наилучшего результата нужно подобрать параметры для CatBoostRegressor. (Параметры были подобраны итеративно)

In [ ]:

# Оптимальные параметры модели
cbr_params = {
    'depth': 4,
    'n_estimators': 180,
    'l2_leaf_reg': 13,
    'subsample': 0.7,
    'colsample_bylevel': 0.8,
    'random_state': 1,
    'verbose': 0
}

# Интеративный способ подхода для нахождения оптимальных параметров. Я отсматривала на экране подходящие для меня варианты(было необходимо уменьшить вероятность переобучения и получить ошибку меньше 90)

In [ ]:
# Оптимальные параметры модели
cbr_params = {
    'depth': 4,
    'n_estimators': 180,
    'l2_leaf_reg': 13,
    'subsample': 0.7,
    'colsample_bylevel': 0.8,
    'random_state': 1,
    'verbose': 0
}

depth=[int(i) for i in range(4,10)]
n_estimators=[int(i) for i in range(170,300,10)]
l2_leaf_reg=[int(i) for i in range(6,17)]

l=50
cbr_params_g = {
    'depth': 6,
    'n_estimators': 220,
    'l2_leaf_reg': 8,
    'subsample': 0.7,
    'colsample_bylevel': 0.8,
    'random_state': 1,
    'verbose': 0
}

for i in depth:
    for k in l2_leaf_reg:
    
        for j in n_estimators:
            print(i)
            cbr_params['depth']=i
            cbr_params['n_estimators']=j
            cbr_params['l2_leaf_reg']=k
            
            # P.S. цифры 1,2... я писала, чтобы проверять корректность работы программы :)
            # Создаем модель
            cbr = CatBoostRegressor(**cbr_params)
            print(1)
            # Разделение данных на обучающий и проверочный наборы
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)
            print(2)
            # Обучение модели
            cbr.fit(X_train, y_train, early_stopping_rounds=100)
            print(3)
            # Предсказания
            y_train_pred = cbr.predict(X_train)
            y_val_pred = cbr.predict(X_val)
            print(4)
            # Удаление выбросов
            y_train_wo_outliers = y_train[y_train != 1093].dropna()
            X_train_wo_outliers = X_train.loc[y_train_wo_outliers.index]
            print(5)
        
            # Вычисление MAE
            mae_train = round(mean_absolute_error(y_train, y_train_pred), 2)
            mae_val = round(mean_absolute_error(y_val, y_val_pred), 2)
            
            print("!")
            print(mae_val)
            print(mae_train)
            print(cbr_params)
            print("!")
            
            # Отсчитываем 10%
            mae_train_10_pr=mae_train/100*10
            mae_val_10_pr=mae_val/100*10
            
            if mae_train_10_pr<abs(mae_train-mae_val) and mae_val_10_pr<abs(mae_train-mae_val) and mae_val<100 and mae_train<100:
                if abs(mae_train-mae_val)<l:
                    l=abs(mae_train-mae_val)
                    cbr_params_g=cbr_params
                    print(cbr_params_g)
                    print(mae_val)
                    print(mae_train)
                    
            
print(cbr_params_g)

In [ ]:
# Создаем модель
cbr = CatBoostRegressor(**cbr_params)

# Разделение данных на обучающий и проверочный наборы
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)

# Обучение модели
cbr.fit(X_train, y_train, early_stopping_rounds=100)

# Предсказания
y_train_pred = cbr.predict(X_train)
y_val_pred = cbr.predict(X_val)

# Удаление выбросов
y_train_wo_outliers = y_train[y_train != 1093].dropna()
X_train_wo_outliers = X_train.loc[y_train_wo_outliers.index]
#print(y_train_wo_outliers)
#print(X_train_wo_outliers)

# Вычисление MAE
mae_train = round(mean_absolute_error(y_train, y_train_pred), 2)
mae_val = round(mean_absolute_error(y_val, y_val_pred), 2)
print(f'MAE on train set = {mae_train}')
print(f'MAE on test set = {mae_val}')

# Таким образом была создана модель со значением 88.91<90; Модель без явного переобучения. 